<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Find and Delete RDS (Relational Database Service) Instances with Low CPU Utilization</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Delete-RDS-Instances-with-Low-CPU-Utilization"><u>Delete RDS Instances with Low CPU Utilization</u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1)<a href="#1" target="_self" rel="noopener"> Find RDS Instances with Low CPU Utilization</a><br>2)<a href="#2" target="_self" rel="noopener"> Delete RDS Instances</a></p>

In [ ]:
if region == None:
    region = ''
if db_identifiers and not region:
    raise SystemExit("Provide a region for the RDS Instance names!")

<h3 id="Find-ECS-Clusters-with-Low-CPU-Utilization"><a id="1" target="_self" rel="nofollow"></a>Find RDS Instances with Low CPU Utilization</h3>
<p>Using unSkript's Find RDS Instances with Low CPU Utilization action, we will find instances with a low CPU utilization given a threshold percentage using the&nbsp;<span style="color: rgb(53, 152, 219);">CPUUtilization <span style="color: rgb(0, 0, 0);">attribue found in the statistics list of&nbsp; <em>get_metric_statistics</em> API of Cloudwatch.</span></span></p>
<blockquote>
<p>This action takes the following parameters: <code>region, threshold, duration_minutes</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>low_cpu_instances</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator
import pprint
from datetime import datetime,timedelta


from beartype import beartype
@beartype
def aws_find_rds_instances_with_low_cpu_utilization_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_find_rds_instances_with_low_cpu_utilization(handle, utilization_threshold:int=10, region: str = "", duration_minutes:int=5) -> Tuple:
    """aws_find_rds_instances_with_low_cpu_utilization finds RDS instances that have a lower cpu utlization than the given threshold

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the RDS.

        :type utilization_threshold: integer
        :param utilization_threshold: The threshold percentage of CPU utilization for an RDS Instance.

        :type duration_minutes: integer
        :param duration_minutes: Value in minutes to get the start time of the metrics for CPU Utilization

        :rtype: status, list of instances and their region.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            rdsClient = handle.client('rds', region_name=reg)
            cloudwatchClient = handle.client('cloudwatch', region_name=reg)
            all_instances = aws_get_paginator(rdsClient, "describe_db_instances", "DBInstances")
            for db in all_instances:
                response = cloudwatchClient.get_metric_data(
                    MetricDataQueries=[
                        {
                            'Id': 'cpu',
                            'MetricStat': {
                                'Metric': {
                                    'Namespace': 'AWS/RDS',
                                    'MetricName': 'CPUUtilization',
                                    'Dimensions': [
                                        {
                                            'Name': 'DBInstanceIdentifier',
                                            'Value': db['DBInstanceIdentifier']
                                        },
                                    ]
                                },
                                'Period': 60,
                                'Stat': 'Average',
                            },
                            'ReturnData': True,
                        },
                    ],
                    StartTime=(datetime.now() - timedelta(minutes=duration_minutes)).isoformat(),
                    EndTime=datetime.utcnow().isoformat(),
                )
                if 'Values' in response['MetricDataResults'][0]:
                    cpu_utilization = response['MetricDataResults'][0]['Values'][0]
                    if cpu_utilization < utilization_threshold:
                        db_instance_dict = {}
                        db_instance_dict["region"] = reg
                        db_instance_dict["instance"] = db['DBInstanceIdentifier']
                        result.append(db_instance_dict)
        except Exception as error:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "region",
    "duration_minutes": "int(duration_minutes)",
    "utilization_threshold": "int(threshold)"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not db_identifiers",
    "condition_result": true
    }''')
task.configure(outputName="low_cpu_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_find_rds_instances_with_low_cpu_utilization, lego_printer=aws_find_rds_instances_with_low_cpu_utilization_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Low-CPU-Utilization-RDS-Instances">Create List of Low CPU Utilization RDS Instances</h3>
<p>This action filters regions that have no clusters with low CPU utilization and creates a list of those that have them.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_low_cpu_instances</code></p>
</blockquote>

In [ ]:
all_low_cpu_instances = []
dummy = []
try:
    if low_cpu_instances[0] == False:
        if len(low_cpu_instances[1]) != 0:
            all_lower_ttl_records = low_cpu_instances[1]
except Exception:
    for ins_identifier in db_identifiers:
        data_dict = {}
        data_dict["region"] = region
        data_dict["instance"] = ins_identifier
        all_low_cpu_instances.append(data_dict)
print(all_low_cpu_instances)
task.configure(outputName="all_low_cpu_instances")

<h3 id="Delete-RDS-Instance"><a id="2" target="_self" rel="nofollow"></a>Delete RDS Instance</h3>
<p>This action deletes instances found in Step 1.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters:&nbsp;<code>instance_id, region</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Dict
import pprint

from beartype import beartype
@beartype
def aws_delete_rds_instance_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_delete_rds_instance(handle, region: str, instance_id: str) -> Dict:
    """aws_delete_rds_instance dict of response.

        :type region: string
        :param region: AWS Region.

        :type instance_id: string
        :param instance_id: The DB instance identifier for the DB instance to be deleted. This parameter isn’t case-sensitive.

        :rtype: dict of response of deleting an RDS instance
    """
    try:
        ec2Client = handle.client('rds', region_name=region)
        response = ec2Client.delete_db_instance(DBInstanceIdentifier=instance_id)
        return response
    except Exception as e:
        raise Exception(e)




task = Task(Workflow())
task.configure(continueOnError=False)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "instance_id": "iter.get(\\"instance\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_low_cpu_instances",
    "iter_parameter": ["region","instance_id"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_low_cpu_instances)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_delete_rds_instance, lego_printer=aws_delete_rds_instance_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to filter low CPU utilization RDS Instance given threshold percentage and delete them. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>